In [0]:
# Library to process MIDI files
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Bidirectional, Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv1D,MaxPooling1D, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [0]:
# !pip install mido

In [0]:
import mido
from mido import MidiFile, MidiTrack, Message

In [140]:
tf.__version__

'2.2.0'

### Data Preprocessing

In [141]:
music_list = pd.read_csv('music_list.csv')
music_list = music_list.fillna('Not Available')
music_list

,1-Calming,2-Sports,3-Mysterious,4-Nostalgia,5-Happy
0,Big Sky,Shiatsu,Vague,All That You Will Be,Shadows in Vienna
1,Maybe Another Day,Kermode,Cloak,Fractured Memories,Magic Forest
2,The Human Derivative,Run Till Dawn,Lethal Secrets,When It Rains It Pours,Mistaken by Accident
3,The Crossing,Whatever Man,Remote,Near Her,New Orleans Diner
4,A Presence Felt,Body Work,The Mole,Days,Life in Tinseltown
5,No Good Byes,Space Palace,The Unveiling,Mornings,Harlem Rollout
6,Awash,Black Coffee,Ginger Root,Always the Same,Summertime Memory
7,A Whisper In The Dark,Unleash My Halo,Fiona,Through These Doors,Countryside Living
8,Whisper of the Pines,Way Way Back,Dust Blaster,Heading Home Again,On the Rooftop
9,Traces,The Sooner The Better,Fighter Jet,A Lighthouse In Space,The Upper Hand


In [0]:
# find a filename that contains a string

# import os

# folder_name = '1-Calming'
# keyword = 'A Presence Felt'
# for fname in os.listdir('midi/'+folder_name):
#     if keyword in fname:
#         print(fname, "has the keyword")

In [0]:
# prepare a dictionary for song names for each category

# import os
# song_dict = {}
# for folder_name in music_list.columns:
# #     print(folder_name, '\n')
#     song_dict[folder_name] = []
#     for fname in os.listdir('midi/'+folder_name):
# #         print('fname: ', fname)
#         song_dict[folder_name].append(fname)

# print(song_dict['1-Calming'])

In [0]:
# song_dict.keys()

In [0]:
# X = []
# y = []
# for cat in song_dict.keys():
#     for fname in song_dict[cat]:
#         if fname != '.DS_Store':
#             y.append(cat)
#     #         midi_filename = 'midi/1-calming/ES_A Presence Felt - Gavin Luke.mp3.mid'
#             midi_filename = 'midi/' + cat + '/' + fname
# #             print(midi_filename)
#             midi = MidiFile(midi_filename)
#             cnt = 0
#             for msg in midi:
#                 print(msg)
#                 cnt+=1
#                 if cnt>100:
#                     break 

#             # save notes for the song as a list
#             notes = []
#             for msg in midi:
#                 if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
#                     data = msg.bytes()
#                     notes.append(data[1]) # we are just adding the note!

#             ## rescale all the notes to the [0,1] interval
#             scaler = MinMaxScaler(feature_range=(0,1))
#             scaler.fit(np.array(notes).reshape(-1,1))
#             notes = list(scaler.transform(np.array(notes).reshape(-1,1)))
#             notes = [list(note)[0] for note in notes]

#             X.append(notes)
        
# print(X)
# print(y)


In [0]:
# df_X = pd.DataFrame(X)
# df_X.to_csv('df_X.csv', index=False)
# df_y = pd.DataFrame(y)
# df_y.to_csv('df_y.csv', index=False)

In [147]:
df_X = pd.read_csv('df_X.csv')
df_X = df_X.fillna(0)
df_X.shape

(50, 44841)

In [148]:
df_y = pd.read_csv('df_y.csv')
df_y.shape

(50, 1)

In [0]:
X_read = np.array([df_X.iloc[i,:].tolist() for i in range(df_X.shape[0])])

In [150]:
X_read.shape

(50, 44841)

In [0]:
y = df_y['0'].tolist()

In [152]:
# generate dummy variable y

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
encoded_Y
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y.shape

(50, 5)

#### Data Augmentation



In [0]:
n_col = df_X.shape[1]
len_seq = 800

In [154]:
n_col / len_seq

56.05125

In [155]:
n_col//len_seq

56

In [0]:
n_col = df_X.shape[1]
# len_seq = 1000

X_aug = []
y_aug = []

for i_row in range(df_y.shape[0]):
  song_data = df_X.iloc[i_row,:]
  # print(song_data)
  # print(song_data.shape)
  for i_seg in range((n_col//len_seq)):
    col_start = i_seg * len_seq
    # print(col_start)
    col_end = (i_seg + 1) * len_seq
    # print(col_end)
    seg = song_data.iloc[col_start:col_end]
    if sum(seg) == 0.0:
      continue
    else:
      X_aug.append(seg)
      y_aug.append(df_y.iloc[i_row])
# print(X_aug)
# print(y_aug)


In [157]:
np.shape(X_aug)

(835, 800)

In [158]:
np.shape(y_aug)

(835, 1)

In [159]:
unique, counts = np.unique(y_aug, return_counts=True)
dict(zip(unique, counts))

{'1-Calming': 69,
 '2-Sports': 303,
 '3-Mysterious': 219,
 '4-Nostalgia': 82,
 '5-Happy': 162}

In [0]:
# X_read = np.array([df_X.iloc[i,:].tolist() for i in range(df_X.shape[0])])

X_read = np.array(X_aug)


In [161]:
# generate dummy variable y

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_aug)
encoded_Y = encoder.transform(y_aug)
encoded_Y
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(835, 5)

### Define Parameters

In [0]:
# n_prev = 10000

n_prev = len_seq
test_size = 0.2

### Neural Network

In [0]:
# split datasets into train and test
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X_read, dummy_y, test_size = test_size, 
                                                    random_state = 0)

In [0]:
# set the length of all sequences to be equal to max_len

max_len = 0
temp_len = 0
for i in range(len(X_read)):
    max_len = len(X_read[i])
#     print(max_len)
    if max_len < temp_len:
        max_len = temp_len
    else:
        temp_len = max_len
max_len

# X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_train_nn = X_train_nn.reshape((X_train_nn.shape[0], X_train_nn.shape[1],1))

# X_test = sequence.pad_sequences(X_test, maxlen=max_len)
X_test_nn = X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1)


#### Only LSTM + Dense Layer

In [165]:
X_train_nn.shape

(668, 800, 1)

In [166]:
# Define a model
# n_prev = 15000

model = Sequential()
# input_shape = (n_prev, 1)


model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])


print(model.summary())

# train model
# model.fit(np.array(X_train_nn[:,:n_prev]), np.array(y_train_nn), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])

model.fit(np.array(X_train_nn), np.array(y_train_nn), 
          32, 50, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_24 (Bidirectio (None, 800, 512)          528384    
_________________________________________________________________
dropout_24 (Dropout)         (None, 800, 512)          0         
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 800, 256)          656384    
_________________________________________________________________
dropout_25 (Dropout)         (None, 800, 256)          0         
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dropout_26 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)              

In [167]:
pred_proba = model.predict_proba(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1))

for i in range(len(y_test_nn)):
  print('Probability Prediction: ',pred_proba[i,:], '; Real Target: ', y_test_nn[i])


Instructions for updating:
Please use `model.predict()` instead.
Probability Prediction:  [0.22618812 0.9274609  0.7108468  0.26831067 0.53153265] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.22787663 0.92656267 0.71018213 0.26917246 0.53176874] ; Real Target:  [0. 0. 0. 1. 0.]
Probability Prediction:  [0.22734457 0.92684376 0.71035826 0.2689138  0.53172076] ; Real Target:  [1. 0. 0. 0. 0.]
Probability Prediction:  [0.22608697 0.92750674 0.71087307 0.268264   0.5315097 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.22662845 0.9272368  0.71068084 0.26850802 0.53162014] ; Real Target:  [0. 0. 1. 0. 0.]
Probability Prediction:  [0.22609693 0.92752725 0.7109147  0.26821288 0.5315445 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.22625923 0.92745686 0.71089685 0.26828384 0.53156275] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [0.22619072 0.92746985 0.7108665  0.26827252 0.53154904] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Pre

In [168]:
# X_test_trunc = X_test_nn[:, :n_prev]
# scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
#                         y_test_nn, verbose=1)


scores = model.evaluate(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1),
                        y_test_nn, verbose=1)

print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

6/6 [==============================] - 12s 2s/step - loss: 1.4736 - accuracy: 0.3772
Out-of-Sample Accuracy: 37.72%


#### Conv1d + LSTM + Dense Layers

In [169]:
# Define a model
n_prev = 15000
embedding_vector_length = 300
# max_len = 1000

model = Sequential()
# model.add(Embedding(n_prev, embedding_vector_length, input_length=max_len))

model.add(Conv1D(filters=256, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(Bidirectional(LSTM(256 , return_sequences=True), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))


model.add(Conv1D(filters=128, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(Bidirectional(LSTM(128,  return_sequences=True), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))

model.add(Conv1D(filters=64, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))

# model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
# model.add(Dropout(0.3))

model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

# train model
# model.fit(np.array(X_train_nn[:,:n_prev]), np.array(y_train_nn), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])

model.fit(np.array(X_train_nn), np.array(y_train_nn), 
          32, 50, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 14993, 256)        2304      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3748, 256)         0         
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 3748, 512)         1050624   
_________________________________________________________________
dropout_27 (Dropout)         (None, 3748, 512)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3741, 128)         524416    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 935, 128)          0         
_________________________________________________________________
bidirectional_28 (Bidirectio (None, 935, 256)        

In [170]:
# X_test_trunc = X_test_nn[:, :n_prev]
# scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
#                         y_test_nn, verbose=1)
# print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

pred_proba = model.predict_proba(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1))

for i in range(len(y_test_nn)):
  print('Probability Prediction: ',pred_proba[i,:], '; Real Target: ', y_test_nn[i])
  
scores = model.evaluate(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1),
                        y_test_nn, verbose=1)

print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

Probability Prediction:  [3.3441186e-04 1.5737832e-02 8.3246791e-01 6.1406195e-02 2.8095305e-02] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.00232598 0.6924454  0.0081687  0.00711158 0.05234113] ; Real Target:  [0. 0. 0. 1. 0.]
Probability Prediction:  [0.09716648 0.01950946 0.09123164 0.00255939 0.00096065] ; Real Target:  [1. 0. 0. 0. 0.]
Probability Prediction:  [0.13948953 0.00350127 0.12813729 0.85967016 0.5801634 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.13108486 0.01756969 0.17095229 0.01325309 0.00199172] ; Real Target:  [0. 0. 1. 0. 0.]
Probability Prediction:  [3.5050511e-04 1.1606544e-02 1.5301844e-01 7.2316825e-03 6.5621287e-01] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [3.6074817e-03 9.3965524e-01 8.8632107e-04 1.0386966e-04 5.0141513e-03] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [1.0184050e-03 9.4972461e-01 5.3817034e-04 2.1404687e-05 1.6574562e-03] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Predicti

#### Define a different model with more focus on using Conv1D

In [171]:
# Define a model
n_prev = 15000
embedding_vector_length = 300
# max_len = 1000

model = Sequential()
# model.add(Embedding(n_prev, embedding_vector_length, input_length=max_len))

# model.add(Conv1D(filters=128, kernel_size=8, strides=1, activation='relu', 
#                  input_shape=(n_prev, 1)))
# model.add(MaxPooling1D(pool_size=4))

model.add(Conv1D(filters=128, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.3))


model.add(Conv1D(filters=64, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.3))

model.add(Conv1D(filters=32, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(16, return_sequences=False), input_shape=(n_prev, 1)))
model.add(Dropout(0.3))
# model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

# train model
# model.fit(np.array(X_train_nn[:,:n_prev]), np.array(y_train_nn), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])
model.fit(np.array(X_train_nn), np.array(y_train_nn), 
          32, 50, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 14993, 128)        1152      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 3748, 128)         0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 3748, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 3741, 64)          65600     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 935, 64)           0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 935, 64)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 928, 32)         

In [172]:
# X_test_trunc = X_test_nn[:, :n_prev]
# scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
#                         y_test_nn, verbose=0)
# print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

pred_proba = model.predict_proba(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1))

for i in range(len(y_test_nn)):
  print('Probability Prediction: ',pred_proba[i,:], '; Real Target: ', y_test_nn[i])
  
scores = model.evaluate(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1),
                        y_test_nn, verbose=1)

print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

Probability Prediction:  [0.00389868 0.04247266 0.19088599 0.18606126 0.49836013] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.01330325 0.08211595 0.06241238 0.2432146  0.7232635 ] ; Real Target:  [0. 0. 0. 1. 0.]
Probability Prediction:  [0.7151164  0.13188004 0.41061962 0.07714587 0.01219124] ; Real Target:  [1. 0. 0. 0. 0.]
Probability Prediction:  [0.68823636 0.15812334 0.28076416 0.55114466 0.42939556] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.60492766 0.08060792 0.5234299  0.06803763 0.00796729] ; Real Target:  [0. 0. 1. 0. 0.]
Probability Prediction:  [0.00276956 0.04122564 0.20916095 0.17744756 0.5189631 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.00107512 0.519453   0.02002683 0.00114581 0.10566103] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [1.0407865e-03 8.3316892e-01 5.1420629e-03 9.1941751e-05 1.8671900e-02] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [3.9246678e-04 6.0484391e-01 1.7722487e-

### Other Classifiers

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# X = df_X.to_numpy()
# y = df_y.to_numpy()
X = X_aug
y = y_aug

In [0]:
# n_prev = 15000

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, 
                                                    random_state = 0)

X_train_trunc = X_train
X_test_trunc = X_test
# X_train_trunc = X_train[:, :n_prev]
# X_train_trunc = np.array(X_train[:,:n_prev]).reshape(X_train_trunc.shape[0], 
#                                     X_train_trunc.shape[1])
# y_train = np.array(y_train)

# X_test_trunc = X_test[:, :n_prev]
# X_test_trunc = X_test_trunc.reshape(X_test_trunc.shape[0], 
#                                     X_test_trunc.shape[1])

In [0]:
y_test = np.array(y_test)

In [0]:
### Random Forest
from sklearn.ensemble import RandomForestClassifier

In [192]:
rf_clf = RandomForestClassifier(n_estimators = 88,
                                max_depth=2, 
                                min_samples_leaf=2,
                                random_state=0, verbose=0)

param_grid = {
    'n_estimators': [8, 9, 10, 11, 12],
    'max_depth': [1, 2, 3,4],
    'min_samples_leaf': [2,4, 5, 6,7]
}

grid_rf_clf = GridSearchCV(rf_clf, param_grid, cv=3, verbose=0)
grid_rf_clf.fit(X_train_trunc, y_train)

grid_rf_pred_train = grid_rf_clf.predict(X_train_trunc)
grid_rf_pred = grid_rf_clf.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

In [193]:
print('train accuracy: ', accuracy_score(grid_rf_pred_train, y_train), '\n')

print('accuracy: ', accuracy_score(grid_rf_pred, y_test), '\n')

print('parameters: ', grid_rf_clf.best_params_, '\n')

print('testing set: ', y_test.reshape(y_test.shape[1], 
                                      y_test.shape[0]), '\n')
print('prediction: ', grid_rf_pred)

train accuracy:  0.6302395209580839 

accuracy:  0.5209580838323353 

parameters:  {'max_depth': 4, 'min_samples_leaf': 6, 'n_estimators': 12} 

testing set:  [['5-Happy' '4-Nostalgia' '1-Calming' '5-Happy' '3-Mysterious' '5-Happy'
  '2-Sports' '2-Sports' '2-Sports' '3-Mysterious' '1-Calming' '2-Sports'
  '4-Nostalgia' '2-Sports' '3-Mysterious' '3-Mysterious' '3-Mysterious'
  '3-Mysterious' '2-Sports' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '4-Nostalgia' '2-Sports' '5-Happy' '5-Happy' '3-Mysterious' '5-Happy'
  '4-Nostalgia' '4-Nostalgia' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '2-Sports' '2-Sports' '2-Sports' '2-Sports' '1-Calming' '3-Mysterious'
  '3-Mysterious' '3-Mysterious' '1-Calming' '4-Nostalgia' '2-Sports'
  '1-Calming' '4-Nostalgia' '5-Happy' '4-Nostalgia' '2-Sports' '5-Happy'
  '3-Mysterious' '5-Happy' '2-Sports' '5-Happy' '3-Mysterious'
  '4-Nostalgia' '5-Happy' '3-Mysterious' '2-Sports' '2-Sports'
  '4-Nostalgia' '5-Happy' '2-Sports' '3-Mysterious' '2-Sports'
 

In [0]:
### SVM Classifier
from sklearn.svm import SVC

In [194]:
svc = SVC()

param_grid = {
    'kernel': ['poly', 'rbf', 'sigmoid','linear'],
    'C': [5, 10, 15, 30, 100],
    'gamma': [0.1, 0.001, 0.0008, 0.0005]
}

grid_svc = GridSearchCV(svc, param_grid, cv=3, verbose=0)
grid_svc.fit(X_train_trunc, y_train)

grid_svc_pred_train = grid_svc.predict(X_train_trunc)
grid_svc_pred = grid_svc.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [195]:

print('train accuracy: ', accuracy_score(grid_svc_pred_train, y_train), '\n')

print('accuracy: ', accuracy_score(grid_svc_pred, y_test), '\n')
print('parameters: ', grid_svc.best_params_, '\n')

print('testing set: ', y_test.reshape(y_test.shape[1], 
                                      y_test.shape[0]), '\n')
print('prediction: ', grid_svc_pred)

train accuracy:  0.6362275449101796 

accuracy:  0.5029940119760479 

parameters:  {'C': 5, 'gamma': 0.001, 'kernel': 'rbf'} 

testing set:  [['5-Happy' '4-Nostalgia' '1-Calming' '5-Happy' '3-Mysterious' '5-Happy'
  '2-Sports' '2-Sports' '2-Sports' '3-Mysterious' '1-Calming' '2-Sports'
  '4-Nostalgia' '2-Sports' '3-Mysterious' '3-Mysterious' '3-Mysterious'
  '3-Mysterious' '2-Sports' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '4-Nostalgia' '2-Sports' '5-Happy' '5-Happy' '3-Mysterious' '5-Happy'
  '4-Nostalgia' '4-Nostalgia' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '2-Sports' '2-Sports' '2-Sports' '2-Sports' '1-Calming' '3-Mysterious'
  '3-Mysterious' '3-Mysterious' '1-Calming' '4-Nostalgia' '2-Sports'
  '1-Calming' '4-Nostalgia' '5-Happy' '4-Nostalgia' '2-Sports' '5-Happy'
  '3-Mysterious' '5-Happy' '2-Sports' '5-Happy' '3-Mysterious'
  '4-Nostalgia' '5-Happy' '3-Mysterious' '2-Sports' '2-Sports'
  '4-Nostalgia' '5-Happy' '2-Sports' '3-Mysterious' '2-Sports'
  '3-Mysterious' '3

In [0]:
### Nearest neighbors
from sklearn.neighbors import KNeighborsClassifier

In [205]:
neigh = KNeighborsClassifier()
param_grid = {'n_neighbors': [1,2, 3,4, 5]}
grid_neigh = GridSearchCV(neigh, param_grid, cv=3, verbose=0)

grid_neigh = grid_neigh.fit(X_train_trunc, y_train)

grid_neigh_pred_train = grid_neigh.predict(X_train_trunc)
grid_neigh_pred = grid_neigh.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when

In [206]:
print('train accuracy: ', accuracy_score(grid_neigh_pred_train, y_train), '\n')

print('accuracy: ', accuracy_score(grid_neigh_pred, y_test), '\n')
print('parameters: ', grid_neigh.best_params_, '\n')

train accuracy:  0.7514970059880239 

accuracy:  0.2634730538922156 

parameters:  {'n_neighbors': 2} 



In [0]:
### Multivariate Logistic regression
from sklearn.linear_model import LogisticRegression

In [200]:
logit = LogisticRegression()
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],
    'C': [0.001, 0.005, 0.008, 0.01, 0.1, 0.5]
}

grid_logit = GridSearchCV(logit, param_grid, cv=3, verbose=0)
grid_logit.fit(X_train_trunc, y_train)

grid_logit_pred_train = grid_logit.predict(X_train_trunc)
grid_logit_pred = grid_logit.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Valu

In [201]:

print('train accuracy: ', accuracy_score(grid_logit_pred_train, y_train), '\n')


print('accuracy: ', accuracy_score(grid_logit_pred, y_test), '\n')
print('parameters: ', grid_logit.best_params_, '\n')

train accuracy:  0.5299401197604791 

accuracy:  0.5089820359281437 

parameters:  {'C': 0.005, 'penalty': 'l2'} 



In [0]:
### Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
# X_train_trunc.shape
# y_train.shape

In [0]:
gbc = GradientBoostingClassifier()
# param_grid = {
#     'loss': ['deviance', 'exponential'],
#     'learning_rate': [0.01, 0.05, 0.1, 0.5],
#     'n_estimators': [80, 100, 150, 200, 300],
#     'min_samples_leaf': [1, 2, 5, 8, 10],
#     'max_depth': [3,5,8,12,15]
# }

param_grid = {
    'loss': ['deviance', 'exponential'],
    'learning_rate': [0.005, 0.01, 0.05],
    'n_estimators': [80, 150, 200],
    'min_samples_leaf': [5, 8, 10],
    'max_depth': [3,8,15]
}

grid_gbc = GridSearchCV(gbc, param_grid, cv=3, verbose=0)
grid_gbc.fit(X_train_trunc, y_train)

gbc_pred_train = gbc.predict(X_train_trunc)
grid_gbc_pred = grid_gbc.predict(X_test_trunc)

# gbc = GradientBoostingClassifier()
# gbc.fit(X_train_trunc, y_train)

# gbc_pred_train = gbc.predict(X_train_trunc)

# gbc_pred = gbc.predict(X_test_trunc)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

In [0]:


print('train accuracy: ', accuracy_score(gbc_pred_train, y_train), '\n')

print('accuracy: ', accuracy_score(gbc_pred, y_test), '\n')
# print('parameters: ', gbc.best_params_, '\n')
print('parameters: ', gbc.get_params, '\n')